# PDF 링크 추출

카카오 & 3,4,5 페이지에 해당하는 PDF 링크 모두 추출하기

In [9]:
from pykrx import stock
import pandas as pd

# KOSPI + KOSDAQ 전체
df = stock.get_market_ticker_list(market="ALL")  # 전체 종목 코드 리스트
df_info = pd.DataFrame({
    "종목코드": df,
    "회사명": [stock.get_market_ticker_name(code) for code in df]
})

# 매핑 딕셔너리
name2code = dict(zip(df_info['회사명'], df_info['종목코드']))
code2name = dict(zip(df_info['종목코드'], df_info['회사명']))

def get_name_code(input_val):
    input_val = input_val.strip()
    if input_val.isdigit() and len(input_val) == 6:
        # 종목코드로 이름 찾기
        item_name = code2name.get(input_val)
        if item_name:
            return item_name, input_val
        else:
            print(f"[오류] 종목코드 '{input_val}'에 해당하는 종목명이 없습니다.")
            return "", ""
    else:
        # 종목명으로 코드 찾기
        item_code = name2code.get(input_val)
        if item_code:
            return input_val, item_code
        else:
            print(f"[오류] 종목명 '{input_val}'에 해당하는 종목코드가 없습니다.")
            return "", ""
            
# 사용 예시
user_input = input("종목명 또는 종목코드를 입력하세요: ")
item_name, item_code = get_name_code(user_input) 

if not item_name or not item_code:
    print("입력값을 다시 확인해주세요.")
else:
    print(f"종목명: {item_name}, 종목코드: {item_code}")

# 전체 열 너비 제한 해제
pd.set_option("display.max_colwidth", None)

# 기본 URL과 타겟 페이지 목록
base_url = "https://finance.naver.com"
pages = list(range(1, 5)) 

# 결과 저장용 리스트
pdf_links = []

# 페이지별 크롤링
for page in pages:
    url = f"{base_url}/research/company_list.naver?searchType=itemCode&itemName={item_name}&itemCode={item_code}&page={page}"
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "html.parser")

    for a_tag in soup.find_all("a", href=True):
        href = a_tag['href']
        if href.endswith(".pdf"):
            full_link = urljoin(base_url, href)
            pdf_links.append(full_link)

# DataFrame으로 정리
df = pd.DataFrame(pdf_links, columns=["PDF 링크"])
print(df)


종목명 또는 종목코드를 입력하세요:  몰라


[오류] 종목명 '몰라'에 해당하는 종목코드가 없습니다.
입력값을 다시 확인해주세요.
Empty DataFrame
Columns: [PDF 링크]
Index: []


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin

# 전체 열 너비 제한 해제
pd.set_option("display.max_colwidth", None)

# 기본 URL과 타겟 페이지 목록
base_url = "https://finance.naver.com"
pages = [3, 4, 5]

itemcode =

# 결과 저장용 리스트
pdf_links = []

# 페이지별 크롤링
for page in pages:
    url = f"{base_url}/research/company_list.naver?searchType=itemCode&itemName=카카오&itemCode=035720&page={page}"
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "html.parser")

    for a_tag in soup.find_all("a", href=True):
        href = a_tag['href']
        if href.endswith(".pdf"):
            full_link = urljoin(base_url, href)
            pdf_links.append(full_link)

# DataFrame으로 정리
df = pd.DataFrame(pdf_links, columns=["PDF 링크"])
print(df)


                                                                                PDF 링크
0   https://stock.pstatic.net/stock-research/company/34/20240809_company_235285000.pdf
1   https://stock.pstatic.net/stock-research/company/57/20240809_company_570019000.pdf
2   https://stock.pstatic.net/stock-research/company/18/20240809_company_569300000.pdf
3   https://stock.pstatic.net/stock-research/company/63/20240809_company_109899000.pdf
4    https://stock.pstatic.net/stock-research/company/2/20240809_company_576437000.pdf
..                                                                                 ...
82          https://ssl.pstatic.net/imgstock/upload/research/company/1683519812535.pdf
83          https://ssl.pstatic.net/imgstock/upload/research/company/1683518317748.pdf
84          https://ssl.pstatic.net/imgstock/upload/research/company/1683517856310.pdf
85          https://ssl.pstatic.net/imgstock/upload/research/company/1683514752823.pdf
86          https://ssl.pstatic.net/imgstoc

In [10]:
from naver_report_crawler import NaverReportCrawler

crawler = NaverReportCrawler()
df = crawler.get_pdf_links("카카오", page_from=1, page_to=4) 
print(df)


Page 1에서 30건 누적 추출됨
Page 2에서 58건 누적 추출됨
Page 3에서 88건 누적 추출됨
Page 4에서 116건 누적 추출됨
                                                                                 PDF 링크
0    https://stock.pstatic.net/stock-research/company/64/20250714_company_112234000.pdf
1     https://stock.pstatic.net/stock-research/company/1/20250709_company_467896000.pdf
2        https://stock.pstatic.net/stock-research/company/1/20250709_company_920000.pdf
3    https://stock.pstatic.net/stock-research/company/16/20250702_company_705015000.pdf
4     https://stock.pstatic.net/stock-research/company/1/20250509_company_852217000.pdf
..                                                                                  ...
111          https://ssl.pstatic.net/imgstock/upload/research/company/1699576266087.pdf
112          https://ssl.pstatic.net/imgstock/upload/research/company/1699574035557.pdf
113          https://ssl.pstatic.net/imgstock/upload/research/company/1699572383217.pdf
114          https://ssl.pstatic.net/im

# PDF plumber 사용하여 데이터 전처리

### 라이브러리 설치 및 경고메시지 무시

In [2]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/42.8 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 79.2 MB/s eta 0:00:00


In [3]:
import os
import pdfplumber as pp
import pandas as pd

In [4]:
import logging
logging.getLogger("pdfminer").setLevel(logging.ERROR)

### pdf 파일 불러오기

In [5]:
import os
import requests

# ➊ 저장 폴더 생성
folder_path = "/content/pdf 저장소"
os.makedirs(folder_path, exist_ok=True)

# ➋ 저장된 개수 카운트용 변수
saved_count = 0

# ➌ 첫 번째 컬럼 기준으로 링크 반복
for i, url in enumerate(df.iloc[:, 0], 1):  # 첫 번째 열 기준
    file_name = f"kakao_report_{i}.pdf"
    save_path = os.path.join(folder_path, file_name)

    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(response.content)
            saved_count += 1
    except:
        pass  # 오류는 무시하고 넘어감

# ✅ 마지막에 한 번만 출력
print(f"총 {saved_count}개 저장 완료 ✅")

총 87개 저장 완료 ✅


In [1]:
from naver_report_crawler import NaverReportCrawler

crawler = NaverReportCrawler()
item_name, item_code = crawler.get_name_code("삼성전자")  # 종목명 또는 코드 사용 가능

df_links = crawler.get_pdf_links(item_name, page_from=1, page_to=2)  # PDF 목록 수집
crawler.download_pdfs(df_links, item_name=item_name)  # 폴더명 자동 지정, 다운로드

Page 1에서 30건 누적 추출됨
Page 2에서 58건 누적 추출됨
Page 3에서 88건 누적 추출됨
Page 4에서 116건 누적 추출됨


PDF 다운로드: 100%|███████████████████████████| 116/116 [00:20<00:00,  5.55개/s]


총 116개 저장 완료 ✅ (폴더: 카카오_리포트_pdf/)


In [10]:

!zip -r /tmp/pdf_저장소.zip "/content/pdf 저장소"

# Download the zipped file
from google.colab import files
files.download('/tmp/pdf_저장소.zip')


  adding: content/pdf 저장소/ (stored 0%)
  adding: content/pdf 저장소/kakao_report_56.pdf (deflated 29%)
  adding: content/pdf 저장소/kakao_report_68.pdf (deflated 17%)
  adding: content/pdf 저장소/kakao_report_34.pdf (deflated 35%)
  adding: content/pdf 저장소/kakao_report_37.pdf (deflated 22%)
  adding: content/pdf 저장소/kakao_report_73.pdf (deflated 23%)
  adding: content/pdf 저장소/kakao_report_32.pdf (deflated 18%)
  adding: content/pdf 저장소/kakao_report_6.pdf (deflated 32%)
  adding: content/pdf 저장소/kakao_report_52.pdf (deflated 18%)
  adding: content/pdf 저장소/kakao_report_33.pdf (deflated 20%)
  adding: content/pdf 저장소/kakao_report_14.pdf (deflated 26%)
  adding: content/pdf 저장소/kakao_report_2.pdf (deflated 25%)
  adding: content/pdf 저장소/kakao_report_65.pdf (deflated 22%)
  adding: content/pdf 저장소/kakao_report_43.pdf (deflated 18%)
  adding: content/pdf 저장소/kakao_report_47.pdf (deflated 36%)
  adding: content/pdf 저장소/kakao_report_87.pdf (deflated 26%)
  adding: content/pdf 저장소/kakao_report_12.pdf (d

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
import pdfplumber

# PDF 경로 설정
file_path = "/content/pdf 저장소/kakao_report_1.pdf"

# PDF 열기
with pdfplumber.open(file_path) as pdf:
    # 전체 페이지 순회하며 텍스트 추출
    for i, page in enumerate(pdf.pages):
        text = page.extract_text()
        print(f"\n--- Page {i+1} ---\n")
        print(text)


--- Page 1 ---

Company Report
카카오
035720
Aug 9, 2024
Buy 톡비즈 성장 재가속 필요
유지
TP 51,000원 하향
2Q24 Review: 매출 & 영익 컨센서스 부합
2Q24 매출은 2조 49억원(YoY +4%), 영익은 1,340억원으로 시장 예상치(매출 2조 511억
Company Data
원, 영익 1,332억원)에 부합. 톡비즈 광고 매출은 톡채널 활성 광고주 수 및 전체 친구 수
현재가(08/08) 38,450 원
액면가(원) 100 원 확대로 메시지 비즈니스 성장(YoY +16%), 친구탭 비즈보드와 오픈채팅탭 채팅목록형 광
52주 최고가(보통주) 61,100 원
고 매출 증가에 따른 DA(YoY +5%) 성장으로 3,073억원(YoY +9%) 기록. 거래형 매출은
52주 최저가(보통주) 36,300 원
KOSPI (08/08) 2,556.73p 전 분기와 유사한 정도의 커머스 통합거래액 성장 이어지며 2,066억원(YoY +5%). 플랫폼
KOSDAQ (08/08) 745.28p
자본금 445 억원 기타 모빌리티 전 부문 견조 성장과 페이 결제 및 금융서비스 성장으로 3,535억원(YoY
시가총액 170,504 억원
+18%) 기록. 게임 매출은 기존 게임 매출 하향으로 2,335억원(YoY -13%), 뮤직 매출은
발행주식수(보통주) 44,349 만주
발행주식수(우선주) 0 만주 주요 소속 아티스트 신보 발매와 활발한 콘서트 활동으로 5,109억원(YoY +6%) 기록.
평균거래량(60일) 151.9 만주
평균거래대금(60일) 604 억원
영업비용 중 인건비는 보수적인 인력 채용 기조 유지로 YoY +2%, 매출연동비는 매출 성
외국인지분(보통주) 26.74%
주요주주 장 추이에 따라 YoY +2%, 매출 대비 매출연동비 비중은 39.7%. 외주인프라비는 브레인
김범수 외 92 인 24.22%
MAXIMO PTE 6.31% 영업양수도에 따른 인프라수수료 감소로 YoY -5%, 상각비는 자체, 신규임차

In [7]:
# 1페이지만 보기
file_path = "/content/pdf 저장소/kakao_report_1.pdf"  # 네가 저장한 PDF 경로

with pdfplumber.open(file_path) as pdf:
    first_page = pdf.pages[0]             # 1페이지는 인덱스 0
    text = first_page.extract_text()      # 텍스트 추출
    print(text)

Company Report
카카오
035720
Aug 9, 2024
Buy 톡비즈 성장 재가속 필요
유지
TP 51,000원 하향
2Q24 Review: 매출 & 영익 컨센서스 부합
2Q24 매출은 2조 49억원(YoY +4%), 영익은 1,340억원으로 시장 예상치(매출 2조 511억
Company Data
원, 영익 1,332억원)에 부합. 톡비즈 광고 매출은 톡채널 활성 광고주 수 및 전체 친구 수
현재가(08/08) 38,450 원
액면가(원) 100 원 확대로 메시지 비즈니스 성장(YoY +16%), 친구탭 비즈보드와 오픈채팅탭 채팅목록형 광
52주 최고가(보통주) 61,100 원
고 매출 증가에 따른 DA(YoY +5%) 성장으로 3,073억원(YoY +9%) 기록. 거래형 매출은
52주 최저가(보통주) 36,300 원
KOSPI (08/08) 2,556.73p 전 분기와 유사한 정도의 커머스 통합거래액 성장 이어지며 2,066억원(YoY +5%). 플랫폼
KOSDAQ (08/08) 745.28p
자본금 445 억원 기타 모빌리티 전 부문 견조 성장과 페이 결제 및 금융서비스 성장으로 3,535억원(YoY
시가총액 170,504 억원
+18%) 기록. 게임 매출은 기존 게임 매출 하향으로 2,335억원(YoY -13%), 뮤직 매출은
발행주식수(보통주) 44,349 만주
발행주식수(우선주) 0 만주 주요 소속 아티스트 신보 발매와 활발한 콘서트 활동으로 5,109억원(YoY +6%) 기록.
평균거래량(60일) 151.9 만주
평균거래대금(60일) 604 억원
영업비용 중 인건비는 보수적인 인력 채용 기조 유지로 YoY +2%, 매출연동비는 매출 성
외국인지분(보통주) 26.74%
주요주주 장 추이에 따라 YoY +2%, 매출 대비 매출연동비 비중은 39.7%. 외주인프라비는 브레인
김범수 외 92 인 24.22%
MAXIMO PTE 6.31% 영업양수도에 따른 인프라수수료 감소로 YoY -5%, 상각비는 자체, 신규임차 데이터센터
운영 및 AI 사업

In [8]:
# 1페이지만 보기
file_path = "/content/pdf 저장소/kakao_report_2.pdf"  # 네가 저장한 PDF 경로

with pdfplumber.open(file_path) as pdf:
    first_page = pdf.pages[0]             # 1페이지는 인덱스 0
    text = first_page.extract_text()      # 텍스트 추출
    print(text)

2024년 8월 9일 I 기업분석_Earnings Review
카카오
BUY (유지) (035720)
하반기 변화 예고
목표주가(12M) 60,000원
현재주가(8.08) 38,450원
Key Data 2Q24 Re: 비즈니스 메시지가 이끈 본업
KOSPI 지수 (pt) 2,556.73
52주 최고/최저(원) 61,100/36,300
카카오는 2분기 연결 기준 영업수익 2조 49억원(+4.2%YoY, +0.8%QoQ), 영업이익 1,340
시가총액(십억원) 17,050.4
시가총액비중(%) 0.81 억원(+18.5%YoY, +11.4%QoQ, OPM 6.7%)으로 컨센서스에 부합하는 실적을 기록했다. 비
발행주식수(천주) 443,491.8
60일 평균 거래량(천주) 1,292.7 우호적인 경기 상황에도 톡비즈 광고의 견조한 성장이 나타났다. 2분기 플랫폼 부문 매출액
60일 평균 거래대금(십억원) 54.1
외국인지분율(%) 26.79 은 9,548억원(+12.6%YoY, -0.2%QoQ)이다. 톡비즈 매출액은 5,139억원(+7.0%YoY, -
주요주주 지분율(%)
1.6%QoQ)를 기록했다. 광고 부문에서 비즈보드(+5%YoY)의 성장은 소폭 둔화되었으나 타
김범수 외 92 인 24.22
MAXIMO PTE 6.31
겟팅에 특화된 비즈니스 메시지(+16%YoY)는 여전히 높은 성장률을 보이고 있다. 톡채널
Consensus Data 활성 광고주 수와 전체 친구 수가 +11% 증가했고, 이미지/동영상이 발송 가능한 프리미엄
2024 2025
매출액(십억원) 8,410.5 9,216.3 상품 채택률이 높아져 P가 증가한 영향이다. 영업비용은 1조 8,709억원(+3.4%YoY,
영업이익(십억원) 586.6 730.8
+0.2%QoQ)을 기록했다. 일본 내 경쟁 심화로 픽코마의 마케팅비가 증가했으나 브레인 영
순이익(십억원) 386.3 543.1
EPS(원) 849 1,092 업양수도에 따른 인프라수수료의 일시적인 감소로 안정적인 수준을 유지했다.
BPS(원) 2

다음 스탭

In [9]:
# 데이터 프레임 만들 것임.

# 2024 <- 1페이지에서 가장 위에 이거 포함되어있는 텍스트 추출해서 "date"라는 칼럼에 저장
#  "buy" "hold" "sell" 텍스트 있는 (대문자도 포함) 그 줄 추출해서 "투자의견"
# TP 혹은 목표주가 목표 주가 라는 텍스트 있으면 그 줄 추출해서 "목표주가"
# "증권" 포함된 단어 출력해서 "증권사"

# 이렇게 저장해서 하나의 행을 만들고 이걸 저기있는 모든 PDF 대상으로 실행해줘


In [11]:
import os
import pdfplumber
import pandas as pd
import re

# PDF 폴더 경로 설정
pdf_dir = "/content/pdf 저장소"
os.makedirs(pdf_dir, exist_ok=True)

# 결과 저장용 리스트
data = []

# 폴더 내 PDF 파일 반복
for file_name in sorted(os.listdir(pdf_dir)):
    if not file_name.endswith(".pdf"):
        continue

    file_path = os.path.join(pdf_dir, file_name)

    try:
        with pdfplumber.open(file_path) as pdf:
            # 1페이지 텍스트 추출
            first_page = pdf.pages[0]
            text = first_page.extract_text()
            if not text:
                continue

            lines = text.split('\n')

            # ① 날짜 추출: '2024' 또는 '2023' 포함된 가장 첫 번째 줄
            date = next((line.strip() for line in lines if "2024" in line or "2023" in line), "")

            # ② 투자의견 추출: 'buy', 'hold', 'sell' (대소문자 무시)
            opinion = next((line.strip() for line in lines if re.search(r'\b(buy|hold|sell|매수|매도|유지)\b', line, re.IGNORECASE)), "")

            # ③ 목표주가 추출: 'TP' 또는 '목표주가' 또는 '목표 주가'
            target_price = next((line.strip() for line in lines if any(keyword in line for keyword in ["TP", "목표주가", "목표 주가"])), "")

            # ④ 증권사 추출: '증권'이 포함된 단어 (텍스트 전체에서)
            brokerage_match = re.search(r'\S*증권\S*', text)
            brokerage = brokerage_match.group() if brokerage_match else ""

            # 결과 저장
            data.append({
                "파일명": file_name,
                "date": date,
                "투자의견": opinion,
                "목표주가": target_price,
                "증권사": brokerage
            })

    except Exception as e:
        print(f"[오류] {file_name}: {e}")

# 데이터프레임 생성
df1 = pd.DataFrame(data)
df1

,파일명,date,투자의견,목표주가,증권사
0,kakao_report_1.pdf,"Aug 9, 2024",Buy 톡비즈 성장 재가속 필요,"TP 51,000원 하향",
1,kakao_report_10.pdf,2024-07-16,,"목표주가: 62,000 원(하향)",SK증권에서
2,kakao_report_11.pdf,"Jul 11, 2024",Buy 견조한 톡비즈 vs 제한적인 컨텐츠 성장,"TP 60,000원 하향",
3,kakao_report_12.pdf,2024년 7월 9일 I 기업분석_Earnings Preview,BUY (유지) (035720),"목표주가(12M) 60,000원(하향)",
4,kakao_report_13.pdf,2024년 7월 3일,Buy,"목표주가(하향): 60,000원",
...,...,...,...,...,...
82,kakao_report_84.pdf,투자에 따라 상각비 및 외주인프라비 증가가 이를 상쇄하면서 2023년 영업이익은 2022년,Buy,"TP 74,000원 하향",
83,kakao_report_85.pdf,2023년 5월 8일 I 기업분석_Earnings Review,BUY 1Q23 Re: 투자국면,"목표주가(12M) 80,000원(하향)",
84,kakao_report_86.pdf,2023.5.8,Buy,"목표주가 (유지) 75,000원 경기 불확실성 확대로 광고주들의 보수적 광고 집행으로 비즈보드를 포함한 디스플레이",이베스트투자증권
85,kakao_report_87.pdf,2023년 5월 8일,Buy,"목표주가(하향): 75,000원",


In [17]:
csv_path = "/content/kakao_report_summary.csv"
df1.to_csv(csv_path, index=False, encoding='utf-8-sig')

/tmp/ipython-input-17-4040075441.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df1 = df1.applymap(lambda x: np.nan if x == "" else x)


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://finance.naver.com/research/company_list.naver"
params_template = {
    "searchType": "itemCode",
    "itemName": "카카오",
    "itemCode": "035720",
}

all_data = []

for page in range(3, 6):
    params = params_template.copy()
    params["page"] = page
    response = requests.get(base_url, params=params)
    soup = BeautifulSoup(response.content, "html.parser")

    rows = soup.select("table.type_1 tr")[2:]  # 상위 2줄은 header와 blank row
    for idx, row in enumerate(rows):
        cols = row.find_all("td")
        if len(cols) < 6:
            continue  # 빈줄 등 예외 처리

        stock_name = cols[0].text.strip()
        title = cols[1].text.strip()
        broker = cols[2].text.strip()
        date = cols[4].text.strip()
        views = cols[5].text.strip()

        # 보고서 번호는 순차적으로 kakao_report_1, kakao_report_2 ...
        report_id = f"kakao_report_{len(all_data)+1}"

        all_data.append({
            "종목명": stock_name,
            "제목": title,
            "증권사": broker,
            "첨부": report_id,
            "작성일": date,
            "조회수": views
        })

# 데이터프레임 생성
df_report_info = pd.DataFrame(all_data)

# 예시 출력
import pandas as pd
import IPython.display as disp
disp.display(df_report_info.head())



,종목명,제목,증권사,첨부,작성일,조회수
0,카카오,톡비즈 성장 재가속 필요,교보증권,kakao_report_1,24.08.09,3039
1,카카오,하반기 변화 예고,하나증권,kakao_report_2,24.08.09,4484
2,카카오,다이어트 중,유안타증권,kakao_report_3,24.08.09,2222
3,카카오,매출 성장 보다는 비용 감축에 기대야,유진투자증권,kakao_report_4,24.08.09,1558
4,카카오,"본업, 자회사 모두 성장률 둔화",대신증권,kakao_report_5,24.08.09,1250


In [6]:
import numpy as np

# 증권사 목록 불러오기
broker_list = df_report_info['증권사'].tolist()

# PDF 파일명 만들기
pdf_names = []
pdf_index = 1

for broker in broker_list:
    if broker.strip() == "신한투자증권":
        pdf_names.append(np.nan)
    else:
        pdf_names.append(f'kakao_report_{pdf_index}.pdf')
        pdf_index += 1

# pdf 컬럼 추가
df_report_info["pdf"] = pdf_names
df_report_info

,종목명,제목,증권사,첨부,작성일,조회수,pdf
0,카카오,톡비즈 성장 재가속 필요,교보증권,kakao_report_1,24.08.09,3039,kakao_report_1.pdf
1,카카오,하반기 변화 예고,하나증권,kakao_report_2,24.08.09,4484,kakao_report_2.pdf
2,카카오,다이어트 중,유안타증권,kakao_report_3,24.08.09,2222,kakao_report_3.pdf
3,카카오,매출 성장 보다는 비용 감축에 기대야,유진투자증권,kakao_report_4,24.08.09,1558,kakao_report_4.pdf
4,카카오,"본업, 자회사 모두 성장률 둔화",대신증권,kakao_report_5,24.08.09,1250,kakao_report_5.pdf
...,...,...,...,...,...,...,...
85,카카오,실적 눈높이는 낮추나 하반기 회복기조는 유..,DS투자증권,kakao_report_86,23.05.08,7806,kakao_report_83.pdf
86,카카오,"2분기 컨텐츠, 하반기 톡비즈 반등이 반전 카..",교보증권,kakao_report_87,23.05.08,4774,kakao_report_84.pdf
87,카카오,1Q23 Re: 투자국면,하나증권,kakao_report_88,23.05.08,3500,kakao_report_85.pdf
88,카카오,1Q23 Review: 톡비즈 개편 성과가 중요,이베스트증권,kakao_report_89,23.05.08,1631,kakao_report_86.pdf


In [7]:
import pdfplumber
import pandas as pd
import os
import numpy as np

# PDF 파일 경로
pdf_dir = "/content/pdf 저장소"

# 결과 리스트
opinions = []
targets = []
pdf_files = []

# PDF 순번
pdf_idx = 1

for broker in df_report_info["증권사"]:
    broker = broker.strip()

    # 신한투자증권은 PDF 없음 → NaN 처리
    if broker == "신한투자증권":
        pdf_files.append(np.nan)
        opinions.append(np.nan)
        targets.append(np.nan)
        continue

    # 파일명, 경로 설정
    filename = f"kakao_report_{pdf_idx}.pdf"
    pdf_path = os.path.join(pdf_dir, filename)
    pdf_files.append(filename)

    try:
        with pdfplumber.open(pdf_path) as pdf:
            page = pdf.pages[0]
            lines = page.extract_text().split('\n')
    except Exception as e:
        print(f"Error reading {filename}: {e}")
        opinions.append(np.nan)
        targets.append(np.nan)
        pdf_idx += 1
        continue

    # 투자의견 키워드 탐색
    opinion_keywords = ["BUY", "HOLD", "SELL", "매수", "매도", "유지"]
    opinion_line = next((line for line in lines if any(kw.lower() in line.lower() for kw in opinion_keywords)), np.nan)
    opinions.append(opinion_line)

    # 목표주가 키워드 탐색
    target_keywords = ["TP", "목표주가", "목표 주가", "유지"]
    target_line = next((line for line in lines if any(kw in line for kw in target_keywords)), np.nan)
    targets.append(target_line)

    # 다음 PDF 인덱스로
    pdf_idx += 1

# df_report_info에 칼럼 추가
df_report_info["pdf"] = pdf_files
df_report_info["투자의견"] = opinions
df_report_info["목표주가"] = targets


Cannot set gray non-stroke color because /'P9' is an invalid float value


In [27]:
df_report_info

,종목명,제목,증권사,첨부,작성일,조회수,pdf,투자의견,목표주가
0,카카오,톡비즈 성장 재가속 필요,교보증권,kakao_report_1,24.08.09,3039,kakao_report_1.pdf,Buy 톡비즈 성장 재가속 필요,"TP 51,000원 하향"
1,카카오,하반기 변화 예고,하나증권,kakao_report_2,24.08.09,4484,kakao_report_2.pdf,BUY (유지) (035720),"목표주가(12M) 60,000원"
2,카카오,다이어트 중,유안타증권,kakao_report_3,24.08.09,2222,kakao_report_3.pdf,BUY,"원(YoY +77.6%). 영업이익 기준 시장기대치 부합 목표주가 53,000원 (D)"
3,카카오,매출 성장 보다는 비용 감축에 기대야,유진투자증권,kakao_report_4,24.08.09,1558,kakao_report_4.pdf,BUY,목표주가
4,카카오,"본업, 자회사 모두 성장률 둔화",대신증권,kakao_report_5,24.08.09,1250,kakao_report_5.pdf,BUY,TP 하향. 글로벌 peer 밸류 감소 및 픽코마 지분가치 하향 조정 영향
...,...,...,...,...,...,...,...,...,...
85,카카오,실적 눈높이는 낮추나 하반기 회복기조는 유..,DS투자증권,kakao_report_86,23.05.08,7806,kakao_report_83.pdf,실적 눈높이는 낮추나 하반기 회복기조는 유지 i,"카카오에 대한 투자의견 BUY, 목표주가 8만원을 유지한다. 광고 둔화와 신사업 투 구성중 2차전지·인터넷"
86,카카오,"2분기 컨텐츠, 하반기 톡비즈 반등이 반전 카..",교보증권,kakao_report_87,23.05.08,4774,kakao_report_84.pdf,Buy,"TP 74,000원 하향"
87,카카오,1Q23 Re: 투자국면,하나증권,kakao_report_88,23.05.08,3500,kakao_report_85.pdf,BUY 1Q23 Re: 투자국면,"목표주가(12M) 80,000원(하향)"
88,카카오,1Q23 Review: 톡비즈 개편 성과가 중요,이베스트증권,kakao_report_89,23.05.08,1631,kakao_report_86.pdf,Buy,"목표주가 (유지) 75,000원 경기 불확실성 확대로 광고주들의 보수적 광고 집행으로 비즈보드를 포함한 디스플레이"


In [28]:
# 저장 경로
csv_path = "/content/kakao_report_summary_final.csv"

# CSV로 저장 (한글 깨짐 방지)
df_report_info.to_csv(csv_path, index=False, encoding='utf-8-sig')

# 클래스 가져와서 사용

In [ ]:
from naver_report_crawler import NaverReportCrawler

crawler = NaverReportCrawler()
item_name, item_code = crawler.get_name_code("삼성전자")  # 종목명 또는 코드 사용 가능

df_links = crawler.get_pdf_links(item_name, page_from=1, page_to=2)  # PDF 목록 수집
crawler.download_pdfs(df_links, item_name=item_name)  # 폴더명 자동 지정, 다운로드



In [ ]:
from naver_report_crawler import NaverReportCrawler
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

crawler = NaverReportCrawler()

corp = str(input("기업명 또는 종목코드를 입력하세요: "))
item_name, item_code = crawler.get_name_code(corp)

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

result = []

for i in range(1, 2):  # 1부터 3까지 반복 (1, 2, 3)
    url = f"https://finance.naver.com/research/company_list.naver?keyword=&brokerCode=&writeFromDate=&writeToDate=&searchType=itemCode&itemName={item_name}&itemCode={item_code}&page={i}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    for a_tag in soup.find_all("a", href=True):
        href = a_tag['href']
        text = a_tag.get_text(strip=True)
        if re.match(r"^company_read\.naver\?", href):
            result.append({'링크': href, '제목': text})

# DataFrame으로 보기
df = pd.DataFrame(result)
print(df)


기업명 또는 종목코드를 입력하세요:  005930


                                                   링크  \
0   company_read.naver?nid=84390&page=1&searchType...   
1   company_read.naver?nid=84224&page=1&searchType...   
2   company_read.naver?nid=84220&page=1&searchType...   
3   company_read.naver?nid=84212&page=1&searchType...   
4   company_read.naver?nid=84211&page=1&searchType...   
5   company_read.naver?nid=84208&page=1&searchType...   
6   company_read.naver?nid=84200&page=1&searchType...   
7   company_read.naver?nid=84191&page=1&searchType...   
8   company_read.naver?nid=84189&page=1&searchType...   
9   company_read.naver?nid=84188&page=1&searchType...   
10  company_read.naver?nid=84078&page=1&searchType...   
11  company_read.naver?nid=84041&page=1&searchType...   
12  company_read.naver?nid=84014&page=1&searchType...   
13  company_read.naver?nid=83947&page=1&searchType...   
14  company_read.naver?nid=83773&page=1&searchType...   
15  company_read.naver?nid=83711&page=1&searchType...   
16  company_read.naver?nid=8339

### 증권사, 날짜, 목표가, 투자의견 추출

In [48]:
import requests
from bs4 import BeautifulSoup

test_url_1 = 'https://finance.naver.com/research/company_read.naver?nid=84390&page=1&searchType=itemCode&itemCode=005930'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}
test_response = requests.get(test_url_1, headers=headers)
test_soup = BeautifulSoup(test_response.text, 'html.parser')

# 증권사, 날짜 추출
증권사, 날짜 = None, None
source = test_soup.find('p', class_='source')
if source:
    text = source.get_text('|', strip=True)
    items = [item.strip() for item in text.split('|') if item.strip()]
    if len(items) >= 2:
        securities_firm = items[0]
        date = items[1]
else:
    print("source 태그 없음")

# 목표가 추출
target_price, strong = None, None
div1 = test_soup.find('div', class_='view_info_1')
if div1:
    strong = div1.find('strong')
    if strong:
        target_price = strong.get_text(strip=True)

# 투자의견 추출: money em 다음의 em.coment 찾기
opinion = None
if strong and strong.parent:
    # 부모(em.money)에서 다음에 나오는 em.coment 찾기
    for sib in strong.parent.next_siblings:
        if getattr(sib, 'name', None) == 'em' and 'coment' in sib.get('class', []):
            opinion = sib.get_text(strip=True)
            break

print("증권사:", securities_firm)
print("날짜:", date)
print("목표가:", target_price)
print("투자의견:", opinion)


증권사: 미래에셋증권
날짜: 2025.07.14
목표가: 78,000
투자의견: 매수


In [3]:
from naver_report_crawler import NaverReportCrawler
import pandas as pd

crawler = NaverReportCrawler()
corp_list = ["삼성전자", "카카오"] #종목코드를 넣어도 되고, csv파일에서 뽑고 싶은 목표 주가 종목명이나 종목코드 다 가져와서 돌리면 됨
all_results = []

for corp in corp_list:
    item_name, item_code = crawler.get_name_code(corp)
    df = crawler.get_report_detail_list(item_name, item_code, 1, 1)
    all_results.append(df)  # 리스트에 추가

final_df = pd.concat(all_results, ignore_index=True)
print(final_df)

ModuleNotFoundError: No module named 'naver_report_crawler'

In [ ]:
final_df